In [1]:
!pip install openai neo4j scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 5.3 MB/s eta 0:00:00


In [7]:
pip install sentence-transformers neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 60.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [17]:
from sentence_transformers import SentenceTransformer
from neo4j import GraphDatabase

# Load Hugging Face embedding model (no API key needed)
model = SentenceTransformer('all-MiniLM-L6-v2')

# Connect to Neo4j AuraDB
URI = "neo4j+s://06a8f2c2.databases.neo4j.io"  # Replace with your AuraDB URI
AUTH = ("neo4j", "zbzrSPdJkz0O_pvnpK6K0Xfp61DlZDdpzK0OOxHeqIw")  # Replace with your username/password

# Sample texts
texts = [
    "Machine learning enables computers to learn from data.",
    "Graphs are powerful for representing relationships.",
    "Neo4j is a graph database used for connected data.",
    "Embeddings represent text as dense vectors."
]

# Function to get embedding from sentence-transformers
def get_embedding(text):
    return model.encode(text).tolist()  # convert NumPy array to Python list

# Function to store in Neo4j
def store_text_embedding(tx, text, embedding):
    tx.run(
        """
        CREATE (t:Text {text: $text, embedding: $embedding})
        """,
        text=text,
        embedding=embedding
    )




# Main logic to store everything
with GraphDatabase.driver(URI, auth=AUTH) as driver:
    with driver.session() as session:
        for text in texts:
            emb = get_embedding(text)
            session.write_transaction(store_text_embedding, text, emb)

print("✅ All text embeddings stored in Neo4j successfully.")


/tmp/ipython-input-1747532392.py:41: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(store_text_embedding, text, emb)


✅ All text embeddings stored in Neo4j successfully.


In [18]:

def find_similar_texts(query_text, top_k=3):
    query_emb = model.encode([query_text])[0]
    with driver.session() as session:
        results = session.run("""
            MATCH (t:Text)
            RETURN t.text AS text, gds.similarity.cosine(t.embedding, $queryEmbedding) AS similarity
            ORDER BY similarity DESC LIMIT $topK
        """, queryEmbedding=query_emb.tolist(), topK=top_k)
        return [(record["text"], record["similarity"]) for record in results]


In [19]:
find_similar_texts("machine learning")


/tmp/ipython-input-3388008053.py:3: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


[('Machine learning enables computers to learn from data.',
  0.6405869156238656),
 ('Graphs are powerful for representing relationships.', 0.24143058940574738),
 ('Neo4j is a graph database used for connected data.', 0.20640543000449574)]